# Imports

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import zscore
from sklearn.ensemble import IsolationForest

# 1. Load Data

In [ ]:
df = pd.read_csv(r'Dashboard\E-Building-Data.csv',parse_dates=['date_time'])

In [ ]:
df.head()

## Attributsbeschreibung

date_time := Datum (Format: YYYY-MM-DD HH:MM:SS)\
device_id := CO2-Ampel-ID\
tmp := Durchschnittliche Temperatur\
hum := Durchschnittliche Luftfeuchtigkeit\
CO2 := Durchschnittlicher CO2-Wert\
VOC := Durchschnittliche Schadstoffbelastung\
vis := Durchschnittlicher Lichtwert\
IR := Durchschnittlicher Infrarotwert\
WIFI := Durchschnittliche Anzahl an WiFi-Geräten\
BLE := Durchschnittliche Anzahl an Bluetoothgeräten\
rssi := received signal strength indication, Empfangsfeldstärke kabelloser Kommunikationsanwendungen\
channel_rssi := Gesamtstärke des ganzen Empfangs\
snr := Signal-Rausch-Verhältnis\
gateway := Genutztes Gateway für die Übertragung des Datenpunktes\
channel_index :=\
spreading_factor :=\
bandwidth := Bandbreite ist das Maß für die in einer bestimmten Zeiteinheit über ein Medium übertragene Datenmenge\
f_cnt :=


# 2. Functions

In [ ]:
def detect_and_remove_outliers(dataframe):
    data = dataframe.copy()
    isoforest = IsolationForest(n_estimators = 100, contamination = 0.0075, max_samples = int(data.shape[0]*0.8))
    #Isolation Forest auf den wichtigsten numerischen Werten durchführen (CO2, tmp, vis, hum und VOC).
    prediction = isoforest.fit_predict(data[["CO2", "tmp", "vis", "hum", "VOC"]])
    print("Number of outliers detected: {}".format(prediction[prediction < 0].sum()))
    print("Number of normal samples detected: {}".format(prediction[prediction >= 0].sum()))
    score = isoforest.decision_function(data[["CO2", "tmp", "vis", "hum", "VOC"]])
    data["anomaly_score"] = score
    #Zeilen mit anomaly_score < 0 werden vom Isolation Forest als Ausreißer interpretiert.
    data = data[data.anomaly_score >= 0]
    return data

In [ ]:
def remove_outliers(dataframes, col, return_outliers=False):
    q1 = dataframes[col].quantile(0.25)
    q3 = dataframes[col].quantile(0.75)
    iqr = q3 - q1
    lower_limit = q1 - (1.5 * iqr)
    upper_limit = q3 + (1.5 * iqr)

    if return_outliers:
        # Correct condition to capture outliers
        outliers = dataframes[(dataframes[col] < lower_limit) | (dataframes[col] > upper_limit)]
        return outliers
    else:
        # Correct condition to filter out outliers
        filtered = dataframes[(dataframes[col] >= lower_limit) & (dataframes[col] <= upper_limit)]
        return filtered

In [ ]:
def extract_etage(dataframe):
    dataframe['Etage'] = 'Etage ' + dataframe['room_number'].str[1]
    return dataframe

In [ ]:
def count_outliers(data, col):

    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    iqr = q3 - q1

    lower_boundary = q1 - 1.5*iqr
    upper_boundary = q3 + 1.5*iqr

    number_of_outliers = len(data.loc[(data[col] > upper_boundary) | (data[col] < lower_boundary)])

    return number_of_outliers

In [ ]:
def resample_dataframes(dataframe:pd.DataFrame,resampling_freq:str,resampling_type:str = 'mean',verteilung=False):
    data = dataframe.copy()
    if verteilung:
        data['date_time'] = data['date_time'].dt.weekday
        data = data.groupby('date_time').mean()
        return data

    elif resampling_type == 'min':
        data = data.set_index('date_time').resample(resampling_freq).min().reset_index()
    elif resampling_type == 'max':
        data = data.set_index('date_time').resample(resampling_freq).max().reset_index()
    elif resampling_type == 'mean':
        data = data.set_index('date_time').resample(resampling_freq).mean().reset_index()
    return data

# 3. Datentypen

In [ ]:
df.dtypes

In [ ]:
df = df.astype({
            'snr':'float64',
                                            })

In [ ]:
df.dtypes

# 4. Preprocessing

In [ ]:
features = ['date_time', 'CO2', 'VOC', 'hum', 'tmp', 'vis', 'IR', 'WIFI', 'BLE']

In [ ]:
df.loc[:, "room_number"] = df["device_id"].str.split("-").str[-1]
df.drop("device_id", inplace=True, axis=1)
df = extract_etage(df)

In [ ]:
df.loc[:, "year"] = df["date_time"].dt.year
df.loc[:, "month"] = df["date_time"].dt.month
df.loc[:, "dayofweek"] = df["date_time"].dt.dayofweek
df.loc[:, "hour"] = df["date_time"].dt.hour

In [ ]:
dfe001_with_outliers = df[df["room_number"] == "e001"].copy()
dfe001_with_outliers = resample_dataframes(dfe001_with_outliers[features], "h")
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dfe001_with_outliers.date_time,
    y=dfe001_with_outliers.tmp,
    name='Temperatur',
    marker_color='#64378C'
))
fig.update_layout(
    title='Temperatur vor dem IsolationForest für E001',
    xaxis_title='Datum',
    yaxis_title='Temperatur',
    showlegend=True
)
fig.show()

In [ ]:
dfe001_iso = detect_and_remove_outliers(df)

In [ ]:
df_anomalie_detected = dfe001_iso[dfe001_iso["room_number"] == "e001"].copy()
df_anomalie_detected = resample_dataframes(df_anomalie_detected[features], "h")
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_anomalie_detected.date_time,
    y=df_anomalie_detected.tmp,
    name='Temperatur',
    marker_color='#64378C',
))

fig.update_layout(
    title='Temperatur nach dem IsolationForest für E001',
    xaxis_title='Datum',
    yaxis_title='Temperatur',
    showlegend=True
)
fig.show()

# 5. Anomalies

In [ ]:
unique_rooms = df['room_number'].unique()


anomalies_summary_df = []


for room in unique_rooms:
    room_data = df[df['room_number'] == room].copy()
    
    # Calculate Z-scores for temperature, humidity, CO2, and VOC levels
    room_data['tmp_zscore'] = zscore(room_data['tmp'])
    room_data['hum_zscore'] = zscore(room_data['hum'])
    room_data['CO2_zscore'] = zscore(room_data['CO2'])
    room_data['VOC_zscore'] = zscore(room_data['VOC'])
    
    
    threshold = 3
    
   
    room_data['tmp_anomaly'] = room_data['tmp_zscore'].abs() > threshold
    room_data['hum_anomaly'] = room_data['hum_zscore'].abs() > threshold
    room_data['CO2_anomaly'] = room_data['CO2_zscore'].abs() > threshold
    room_data['VOC_anomaly'] = room_data['VOC_zscore'].abs() > threshold
    
    
    anomalies_summary = {
        'room_number': room,
        'tmp_anomaly': room_data['tmp_anomaly'].sum(),
        'hum_anomaly': room_data['hum_anomaly'].sum(),
        'CO2_anomaly': room_data['CO2_anomaly'].sum(),
        'VOC_anomaly': room_data['VOC_anomaly'].sum()
    }
    
    
    anomalies_summary_df.append(anomalies_summary)

anomalies = pd.DataFrame.from_dict(anomalies_summary_df)

In [ ]:
room_counts = df['room_number'].value_counts().reset_index()
room_counts.columns = ['room_number', 'count']


anomalies = anomalies.merge(room_counts, on='room_number')

# Calculate the anomaly rate
anomalies['anomaly_rate'] = (
    anomalies['tmp_anomaly'] +
    anomalies['hum_anomaly'] +
    anomalies['CO2_anomaly'] +
    anomalies['VOC_anomaly']
) / anomalies['count'].round(4)

anomalies['anomaly_count'] = (
    anomalies['tmp_anomaly'] +
    anomalies['hum_anomaly'] +
    anomalies['CO2_anomaly'] +
    anomalies['VOC_anomaly'] )


anomalies.sort_values(by="anomaly_rate", ascending=False)

In [ ]:
anomalies.sort_values(by="anomaly_rate", ascending=False, inplace=True)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=anomalies.room_number,
    y=anomalies.anomaly_rate,
    name='Anomalierate',
    marker_color='#64378C',

))

fig.update_layout(
    title='Anomalierate pro Raum',
    xaxis_title='Raum',
    yaxis_title='Anomalierate',
    showlegend=True
)

fig.show()

In [ ]:
anomalies.sort_values(by="anomaly_count", ascending=False, inplace=True)
fig = go.Figure()
fig.add_bar(x=anomalies.room_number,y=anomalies.VOC_anomaly, name="VOC anomalies")
fig.add_bar(x=anomalies.room_number,y=anomalies.CO2_anomaly, name="CO2_anomalies")
fig.add_bar(x=anomalies.room_number,y=anomalies.anomaly_count, name="anomalies")
fig.update_layout(barmode="group")
fig.update_layout(
    title='Anomalien mit den größten Anteilen',
    xaxis_title='Raum',
    yaxis_title='Anomalien',
    showlegend=True
)
fig.show()

### CO2-Ampel Feature

In [ ]:
# Farben der CO2-Ampeln laut der Quelle https://www.h-ka.de/fileadmin/Hochschule_Karlsruhe_HKA/Bilder_VW-EBI/HKA_VW-EBI_Anleitung_CO2-Ampeln.pdf
df.loc[(df.CO2 < 850), "color"] = "green"
df.loc[(df.CO2 >= 850) & (df.CO2 < 1200), "color"] = "yellow"
df.loc[(df.CO2 >= 1200) & (df.CO2 < 1600), "color"] = "red"
df.loc[(df.CO2 >= 1600), "color"] = "red_blinking"

In [ ]:
df_ampel = df.sort_values(by="room_number")[["room_number","CO2", "color"]]
df_ampel = df_ampel.merge(room_counts, on='room_number')
df_ampel = df_ampel.groupby(['room_number', 'color']).size().unstack(fill_value=0).reset_index()

In [ ]:
df_ampel["count"] = df_ampel["green"] + df_ampel["red"] + df_ampel["red_blinking"] + df_ampel["yellow"]

In [ ]:
df_ampel["red_blinking_rate"] = df_ampel["red_blinking"] / df_ampel["count"]
df_ampel["red_rate"] =  df_ampel["red"] / df_ampel["count"]
df_ampel["red_yellow_rate"] =  df_ampel["yellow"] / df_ampel["count"]

In [ ]:
df_ampel.sort_values(by="red_yellow_rate", ascending=False, inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_ampel.room_number,y=df_ampel.red_yellow_rate, name="yellow", marker_color="orange"))
fig.add_trace(go.Bar(x=df_ampel.room_number,y=df_ampel.red_rate, name="red", marker_color="red"))
fig.add_trace(go.Bar(x=df_ampel.room_number,y=df_ampel.red_blinking_rate, name="red_blinking", marker_color="black"))
fig.update_layout(barmode="stack")
fig.update_layout(
    title='Räume nach Ampelfarbe "gelb" sortiert',
    xaxis_title='Raum',
    yaxis_title='Rate',
    showlegend=True,
    
)
fig.show()

# 6. Statistische Werte

## Beobachtungen
bandwith hat konstant die gleichen Werte

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

# 7. Correlation

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Assuming datatypes_changed_df is already defined
df_correlation = df.select_dtypes(include='number').iloc[:,:8].corr()

# Create the heatmap
heatmap = go.Heatmap(
    z=df_correlation.values,
    x=df_correlation.columns,  # x-axis labels
    y=df_correlation.index,    # y-axis labels
    colorscale='RdBu',         # similar to 'coolwarm' in seaborn
    zmin=-1,                   # equivalent to vmin
    zmax=1,                    # equivalent to vmax
    colorbar=dict(
        title='Korrelationskoeffizient',
        titleside='right',
        titlefont=dict(size=25),  # Increase title font size
        tickfont=dict(size=25)    # Increase tick font size
    ),
    showscale=True
)

# Add annotations
annotations = []
for i in range(df_correlation.shape[0]):
    for j in range(df_correlation.shape[1]):
        annotations.append(
            go.layout.Annotation(
                text=f"{df_correlation.iloc[i, j]:.2f}",
                x=df_correlation.columns[j],
                y=df_correlation.index[i],
                xref='x1',
                yref='y1',
                showarrow=False,
                font=dict(size=30, color="black", weight='bold')
            )
        )

# Customize the layout
layout = go.Layout(
    xaxis=dict(
        ticks='',
        side='top',
        title_font=dict(size=20),  # Increase x-axis label size
        tickfont=dict(size=25)     # Increase x-axis tick label size
    ),
    yaxis=dict(
        ticks='',
        title_font=dict(size=20),  # Increase y-axis label size
        tickfont=dict(size=25)     # Increase y-axis tick label size
    ),
    width=1200,
    height=1000,
    annotations=annotations
)

# Create the figure
fig = go.Figure(data=[heatmap], layout=layout)

# Display the heatmap
fig.show()


# 8. Anzahl der Datenpunkte pro Raum

In [ ]:
room_value_counts = df.groupby('room_number').count()[['date_time']].sort_values(by='date_time',ascending=False).copy()

In [ ]:
room_value_counts.rename(columns={'date_time':'datapoints'}, inplace=True)

In [ ]:
room_value_counts

In [ ]:
# room_value_counts.to_csv(r'Dashboard\room_value_counts.csv')

In [ ]:
# test = pd.read_csv(r'Dashboard\room_value_counts.csv',index_col='room_number')
# test

In [ ]:

# Convert the dictionary to two lists for plotting
x_values = list(room_value_counts.index)
y_values = list(room_value_counts['datapoints'])

# Create the bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=x_values,
    y=y_values,
    text=y_values,               # Text to display on the bars
    textposition='outside',          # Position the text on top of the bars
    marker_color='#64378C',
    textfont=dict(
        family='Arial',      # Font family
        size=15,             # Font size
        color='black',       # Font color
        weight='bold'        # Font weight (bold)
    )
))
# Customize the layout
fig.update_layout(
    title='Anzahl der Datenpunkte pro Raum',
    xaxis_title='Raum',
    yaxis_title='Anzahl',
    yaxis=dict(
        range=[0, 150000] 
    ) 
)



# Display the bar chart
fig.show()


# 9. Ausreißererkennung

In [ ]:
df_filtered = df[features].copy()

for col in features:
    df_filtered = remove_outliers(df_filtered,col,True)
df_filtered


## Ausreißer

In [ ]:
# Ausreißererkennung mit einem Isolation Forest
df_iso_outliers = detect_and_remove_outliers(df)


In [ ]:
# Ausreißererkennung mit IQR
df_iqr_outliers = df.copy()
for col in features:
    df_iqr_outliers = remove_outliers(df_iqr_outliers,col)

In [ ]:
features

In [ ]:
# df_iso_outliers[['date_time', 'CO2', 'VOC', 'hum', 'tmp', 'vis', 'IR', 'room_number','Etage']].to_csv(r'Dashboard\iso_outliers.csv',index=False)

In [ ]:
df_verteilung_iso = resample_dataframes(df_iso_outliers[features], 'h', verteilung=True)
df_verteilung_iqr = resample_dataframes(df_iqr_outliers[features], 'h', verteilung=True)

In [ ]:
df_verteilung_iso

In [ ]:
df_verteilung_iqr

In [ ]:
df_e001_iso = df_iso_outliers[df_iso_outliers['room_number'] == 'e001'].copy()
df_e001_iqr = df_iqr_outliers[df_iqr_outliers['room_number'] == 'e001'].copy()
df_e001_verteilung_iso = resample_dataframes(df_e001_iso[features], 'h', True)
df_e001_verteilung_iqr = resample_dataframes(df_e001_iqr[features], 'h', True)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
df_plot = df_e001_verteilung_iso.copy()
fig = make_subplots(
        rows=2, cols=3, 
        subplot_titles=[
            "CO2 over time", 
            "VOC over time", 
            "Temperature over time", 
            "Humidity over time", 
            "Visible Light over time", 
            "Infrared over time"
        ]
    )

# Define data columns and subplot positions
plots_info = [
    ('CO2', 1, 3), 
    ('VOC', 2, 1), 
    ('tmp', 1, 1), 
    ('hum', 1, 2), 
    ('vis', 2, 2), 
    ('IR', 2, 3)
]

# Add traces and update axes
for col, row, col_num in plots_info:
    fig.add_trace(
        go.Scatter(x=df_plot.index, y=df_plot[col], mode='lines', name=col),
        row=row, col=col_num
    )
    fig.update_xaxes(title_text="Weekday",  row=row, col=col_num,dtick=1)
    fig.update_yaxes(title_text="Value", row=row, col=col_num)

# Update layout
fig.update_layout(height=800, width=1400, title_text="Isolation Forest - Durchschnittliche Verteilung der Werte pro Wochentag Raum E001", showlegend=False)
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_plot = df_e001_verteilung_iqr.copy()
fig = make_subplots(
        rows=2, cols=3, 
        subplot_titles=[
            "CO2 over time", 
            "VOC over time", 
            "Temperature over time", 
            "Humidity over time", 
            "Visible Light over time", 
            "Infrared over time"
        ]
    )

# Define data columns and subplot positions
plots_info = [
    ('CO2', 1, 3), 
    ('VOC', 2, 1), 
    ('tmp', 1, 1), 
    ('hum', 1, 2), 
    ('vis', 2, 2), 
    ('IR', 2, 3)
]

# Add traces and update axes
for col, row, col_num in plots_info:
    fig.add_trace(
        go.Scatter(x=df_plot.index, y=df_plot[col], mode='lines', name=col),
        row=row, col=col_num
    )
    fig.update_xaxes(title_text="Weekday",  row=row, col=col_num,dtick=1)
    fig.update_yaxes(title_text="Value", row=row, col=col_num)

# Update layout
fig.update_layout(height=800, width=1400, title_text="IQR - Durchschnittliche Verteilung der Werte pro Wochentag Raum E001", showlegend=False)
fig.show()

# Aggregiert nach Zeit

In [ ]:
columns = df.select_dtypes(include='number').iloc[:,:8].columns
overall_outliers = {}
for col in columns:
    overall_outliers[col] = count_outliers(df,col)

In [ ]:



sorted_data = {k: v for k, v in sorted(overall_outliers.items(), key=lambda item: item[1], reverse=True)}
fig = go.Figure()
fig.add_trace(go.Bar(
                x=list(sorted_data.keys()), 
                y=list(sorted_data.values()),
                marker_color='#64378C',
                text=list(sorted_data.values()),
                textposition='outside'))

fig.update_layout(title='Ausreißerverteilung pro Feature',
                  xaxis_title='Feature',
                  yaxis_title='Anzahl',
                  yaxis=dict(range=[0, 170000])
                  )



fig.show()


In [ ]:

hourly_iqr = resample_dataframes(df_iqr_outliers[features],'h')
daily_iqr = resample_dataframes(df_iqr_outliers[features],'d')
weekly_iqr = resample_dataframes(df_iqr_outliers[features],'W')
hourly_iso = resample_dataframes(df_iso_outliers[features],'h')
daily_iso = resample_dataframes(df_iso_outliers[features],'d')
weekly_iso = resample_dataframes(df_iso_outliers[features],'W')

In [ ]:
resampled_iqr_dataframes = [hourly_iqr,daily_iqr,weekly_iqr]
resampled_iso_dataframes = [hourly_iso,daily_iso,weekly_iso]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 3 columns
for df in resampled_iqr_dataframes:
    fig = make_subplots(
        rows=2, cols=3, 
        subplot_titles=[
            "CO2 over time", 
            "VOC over time", 
            "Temperature over time", 
            "Humidity over time", 
            "Visible Light over time", 
            "Infrared over time"
        ]
    )

    # Define data columns and subplot positions
    plots_info = [
        ('CO2', 1, 3), 
        ('VOC', 2, 1), 
        ('tmp', 1, 1), 
        ('hum', 1, 2), 
        ('vis', 2, 2), 
        ('IR', 2, 3)
    ]

    # Add traces and update axes
    for col, row, col_num in plots_info:
        fig.add_trace(
            go.Scatter(x=df['date_time'], y=df[col], mode='lines', name=col),
            row=row, col=col_num
        )
        fig.update_xaxes(title_text="Weekday",  row=row, col=col_num)
        fig.update_yaxes(title_text="Value", row=row, col=col_num)

    # Update layout
    fig.update_layout(height=800, width=1200, title_text="Sensor Readings Over Time", showlegend=False)
    fig.show()



In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=weekly_iso['date_time'],y=weekly_iso['VOC'],mode='lines'))
fig.add_trace(go.Scatter(x=weekly_iso['date_time'],y=weekly_iso['CO2'],mode='lines'))